In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
mode='kaggle'
if mode=='kaggle':
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import psutil
    # Input data files are available in the read-only "../input/" directory
    # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

    import os
    for dirname, _, filenames in os.walk('/kaggle'):
        for filename in filenames:

            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
from sklearn import metrics 
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
# dt_df = dt.fread("/kaggle/working/hist.csv")
# dt_df.to_jay(derivedDataLoc+"hist.jay")
# del [dt_df]

In [ ]:

mode='kaggle'
import os
#import riiideducation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
#from dataExploration import distReports,plotGrabh
#import lightgbm as lgb
import gc
import time
import numpy as np
import sys
import copy
from matplotlib import pyplot
#!{sys.executable} -m pip install datatable --target /kaggle/working/
#sys.path.append("/kaggle/input/packages/")
#import datatable as dt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random

baseLoc="/kaggle/input/"
inputLoc="/kaggle/input/riiid-test-answer-prediction/"
historyLoc="/kaggle/working/"
workingLoc="/kaggle/working/"
derivedDataLoc="/kaggle/working/"

if mode!='kaggle':
    sys.path.append("/home/pooja/PycharmProjects/mlRepository/mlutomation/myCodes/")#availble only for local run
    from funcs import stratified_sample,stratified_sample_report #availble only for local run
    from commonFuncs import packing
    inputLoc="/kaggle/input/"
    historyLoc="/kaggle/working/"
    workingLoc="/kaggle/working/"
    derivedDataLoc="/kaggle/working/"
    historyLoc="/home/pooja/PycharmProjects/Riiid/baseDatasets/history/"


In [ ]:

def stratified_sample(df, strata, size=None, seed=None, keep_index=True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator

    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size / population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry = ''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"

            if s != len(strata) - 1:
                qry = qry + stratum + ' == ' + str(value) + ' & '
            else:
                qry = qry + stratum + ' == ' + str(value)

        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)

    return stratified_df


def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size / population * tmp_grpd['size']).astype(int)
    return tmp_grpd


def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96) ** 2 * 0.5 * 0.5) / 0.02 ** 2)
        n = round(cochran_n / (1 + ((cochran_n - 1) / population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [ ]:
%%time
dtype = {'row_id':'int32','timestamp':'int64', 
         'user_id':'int32' ,
         'content_id':'int16',
         'content_type_id':'int8',
         'answered_correctly':'int8'}
fullMode=False
if True:
    if fullMode:train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', usecols=[0,1, 2, 3, 4, 7], dtype=dtype)#,nrows=6000000)
    else:train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', usecols=[0,1, 2, 3, 4, 7], dtype=dtype,nrows=6000000)
    #train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

    train_df['timestamp']=train_df['timestamp']/1000
    train_df['timestamp']=train_df['timestamp'].astype('int32')
    maxTimestamp=train_df['timestamp'].max()
    #print(maxTimestamp)
    train_df['timestamp']=train_df['timestamp']/87425772
    train_df['timestamp']=train_df['timestamp'].astype('float32')
    gc.collect()
if True:#adding addtional tags
    q=pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")
    len(q['bundle_id'].unique())
    q['tagM']=q['tags'].map(lambda x:str(x).split(" "))
    q['ntag']=q['tagM'].map(lambda x:len(x))
    q['ntag'].max()
    for i in range(q['ntag'].max()):
        q['tag'+str(i)]=q.apply(lambda x:int(x['tagM'][i]) if i<x['ntag'] and x['tagM'][i]!='nan' else 0  ,axis=1).astype('int16')
        #q['tag'+str(i)]=q['tag'+str(i)].fillna(0)
        #print(q['tag'+str(i)].max())
        #print(i)
    q.set_index('question_id',inplace =True)
    
    train_df=train_df.join(q.drop(['tags','tagM','ntag','correct_answer'],axis=1),'content_id')
    for col in ['tag0','tag1','tag2','tag3','tag4','tag5','part','bundle_id']:
        train_df[col]=train_df[col].fillna(0).astype('int16')
print(train_df.memory_usage(deep=True))

In [ ]:
#q=pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")

def createl2conv() :
    l=pd.read_csv("/kaggle/input/riiid-test-answer-prediction/lectures.csv")

    #set(q['question_id']).intersection(set(l['lecture_id']))
    l_list=l['lecture_id']
    #q_list=l['question_id']
    lectureDict1={}
    i=0
    for e in l_list:
        lectureDict1[e]=14000+i
        i+=1
    del [l_list,l]
    return lectureDict1
lectureDict1=  createl2conv() 
def lectureDict(x):
    try : return lectureDict1[x]
    except : return 14498
train_df.loc[train_df[train_df.content_type_id == True].index,'content_id']=train_df[train_df.content_type_id == True]['content_id'].map(lectureDict)
train_df[train_df['content_id']==32736]
train_df[train_df.content_type_id == False].index
train_df['content_id'].max()
del lectureDict1
gc.collect()

In [ ]:
%%time
#external merge
#from other notebook
import pandas as pd
#train_df=train_df1
#train_df=train_df.set_index('row_id')
if False:
    train_df.set_index('row_id',inplace=True)
    train_df.sort_index(inplace=True)
    if fullMode:extra=pd.read_csv("/kaggle/input/sakt-vivekmodified/extra_info.csv",index_col='row_id')#,nrows=6000000)
    else:extra = pd.read_csv("/kaggle/input/sakt-vivekmodified/extra_info.csv",index_col='row_id',nrows=6000000)
    extra.memory_usage(deep=True)
    extra[['relevantLectattended','lect_count']]=extra[['relevantLectattended','lect_count']].astype('int16')
    extra[['content_type_id']]=extra[['content_type_id']].astype('int8')
    extra[['quest_roll_average_50']]=extra[['quest_roll_average_50']].astype('float32').fillna(0.5)
    extra=extra#[extra['content_type_id']==0]

    gc.collect()
    print("used ram:"+str(psutil.virtual_memory().percent))
    #sanity check
    #print((train_df.index-extra.index).unique()[0:10])
    # print(train_df1.shape,extra.shape)
    #print(len(set(train_df.index)-set(extra.index)))
    # extra[extra.index==1835009]
    #train_df1[train_df1.index==1835009]
    #print("remainign ram:"+str(psutil.virtual_memory().percent))
    #spaceDict={}
    for c in ['relevantLectattended','lect_count','quest_roll_average_50']:
        train_df[c]=extra[c]/extra[c].max()
        train_df[c]=train_df[c].astype('float16')
        extra=extra.drop(c,axis=1)
        gc.collect()
        print('dom')
    del extra
    gc.collect()
    #preprocessing extra
    #r['lect_count'].values,r['relevantLectattended'].values,r['quest_roll_average_50'].values

    #train_df['newIndex']=train_df['newIndex'].astype('int8')
    #train_df.set_index('content_type_id',inplace=True)#will chnage
    #train_df=train_df.drop('row_id',axis=1)
    #train_df['content_id']=train_df['content_id'].astype('int16')
    train_df=train_df.reset_index()
    print(train_df.memory_usage(deep=True))
    gc.collect()
    #print(train_df.isna().sum())

    # train_df.shape,extra.shape
    # extra.content_type_id.unique()

In [ ]:
%%time


if False:#tito sampling didn't work as it was was pushing lot of user all rows in validation smale
    train = dt.fread(baseLoc+"baseDatasets/train.jay")
    #train.to_jay(baseLoc+"baseDatasets/train.jay")
    train=train.to_pandas()#[0:1000000]
    #train=train.set_index('row_id')

    user=train.groupby('user_id')['content_id'].count().reset_index()
    user=user#.sample(frac=0.65)
    train=train[train['user_id'].isin(user['user_id'])]
    print(train.shape)
    max_timestamp_u = train[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
    max_timestamp_u.columns = ['user_id', 'max_time_stamp']
    MAX_TIME_STAMP = max_timestamp_u.max_time_stamp.max()

    #(MAX_TIME_STAMP for all users) - (max_time_stamp for each user) is used for this interval.

    def rand_time(max_time_stamp):
        interval = MAX_TIME_STAMP - max_time_stamp
        rand_time_stamp = random.randint(0,interval)
        return rand_time_stamp

    max_timestamp_u['rand_time_stamp'] = max_timestamp_u.max_time_stamp.apply(rand_time)
    train = train.merge(max_timestamp_u, on='user_id', how='left')
    train['viretual_time_stamp'] = train.timestamp + train['rand_time_stamp']
    train = train.sort_values(['viretual_time_stamp'])#.reset_index()

    train=train.drop(['max_time_stamp','rand_time_stamp','viretual_time_stamp'],axis=1)
    train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('bool')
    val_size =int( 0.2*train.shape[0])
    valid = train[-val_size:]
    train = train[:-val_size]
    print("train,valid size:" +str(train.shape[0])+","+str(valid.shape[0]))
    valid=valid[valid['content_type_id']==False]
    train.sort_values(['user_id','timestamp']).to_csv(derivedDataLoc+"train_tito.csv",index=False)
    user=valid.groupby('user_id')['content_id'].count().reset_index()
    user1=user.sample(frac=0.7)
    remUser=user.drop(user1.index,axis=0)#.sample(frac=0.6)
    user2=remUser.sample(frac=0.6)
    user3=remUser.drop(user2.index,axis=0)
    
    seg='user_id'
    count=0
    for ele in [user1,user2,user3]:
        count=count+1
        temp=valid[valid['user_id'].isin(ele['user_id'])]
        temp.sort_values(['user_id','timestamp']).to_csv(derivedDataLoc+str(count)+"valid_tito.csv",index=False)
        print(temp.shape)
        
        
 
    del [train,valid,temp,user1,user2,user3,user,remUser]
    _ = gc.collect()
if True:#creating the user to row mapping
    train= train_df#pd.read_pickle("../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip")
    train=train#[0:1000000]#
    for segregator in ['user_id']:#

        df=train.reset_index()
        train=0
        df=df[['user_id','row_id']]
        first=df.drop_duplicates(subset=['user_id'],keep='first').rename(columns={'row_id':'start_row_id'}).set_index('user_id')
        last=df.drop_duplicates(subset=['user_id'],keep='last').rename(columns={'row_id':'last_row_id'}).set_index('user_id')
        comb=first.join(last)
        comb=comb.sort_values(by=['start_row_id'])#.reset_index()
        comb[['start_row_id','last_row_id']].to_csv(derivedDataLoc+segregator+"_mapping.csv")
        del [train,df,comb,first,last]
        #comb.to_dict(orient='index')
if True:#vivek sampling method
    #step1 :generating sample from user_id and rows to be considered for final sampling
    train= train_df#pd.read_pickle("../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip")
    train=train#[0:1000000]
    train=train.set_index('row_id')#[0:100000]
    er=train[train.content_type_id==0].groupby('user_id')['content_id'].agg(['count'])
    sample=train[train.content_type_id==0].sample(frac=0.15)
    user=sample.groupby('user_id')['content_id'].agg(['count']).rename(columns={'count':'sampleCount'})
    user=user.join(er)
    user['strata']=user['count'].apply(lambda row: row if row<1000 else 1000*int(row/1000))
    #user=user.set_index('user_id')
    dev=stratified_sample(user, strata=['strata'], size=0.8, seed=123,keep_index=True).set_index('user_id')
    valid=stratified_sample(user.drop(dev.index,axis=0), strata=['strata'], size=0.7, seed=123, keep_index=True).set_index('user_id')
    oot=user.drop(dev.index,axis=0).drop(valid.index,axis=0)
    count=1
    del [er,sample,user]
    for d in [dev,valid,oot]:
        temp=d.reset_index().groupby('strata')['sampleCount'].agg(['mean','count']).rename(columns={'count':'count'+str(count),'mean':'mean'+str(count)})#.reset_index().sort_values(by='content_id',ascending=True)
        if count==1: report=temp
        else : report=report.join(temp)
        count=count+1
    report.to_csv(derivedDataLoc+"samplingReport.csv")
    
    var='user_id'
    userMapping=pd.read_csv(derivedDataLoc+var+"_mapping.csv",index_col='user_id')
    final=[]
    count=0
    name=['dev.csv','valid.csv','oot.csv']
    for d in [dev,valid,oot]:#,valid,oot
        rows=[]
        d=d.join(userMapping)
        d['firstRowSample']=d['last_row_id']-d['sampleCount']+1
        d['range']=d.apply(lambda row:range(row['firstRowSample'],row['last_row_id']+1),axis=1)
        startRow=d['firstRowSample'].to_list()
        lastRow=d['last_row_id'].to_list()
        for i in range(0,len(startRow)):
            rows.extend([j for j in range(startRow[i],lastRow[i]+1)])
        train.loc[rows].sort_values(['user_id','timestamp']).to_csv(derivedDataLoc+name[count])
        print(name[count]+str(len(rows)))
        count=count+1
        train=train.drop(rows,axis=0)
    print("hist"+str(train.shape[0]))
    train.sort_values(['user_id','timestamp']).to_csv(derivedDataLoc+'hist.csv')
    del [train,dev,valid,oot,d]#
#     dt_df = dt.fread("/kaggle/working/hist.csv")
#     dt_df.to_jay(derivedDataLoc+"hist.jay")
#     del [dt_df]
        
        
        
    
if True:
    hist579=derivedDataLoc+"hist.csv"
    dev579=derivedDataLoc+"dev.csv"
    valid579=derivedDataLoc+"valid.csv"
    oot579=derivedDataLoc+"oot.csv"
    
    

    

In [ ]:
%%time

if True:
    d=[]
    for ele in [hist579,dev579]:#,valid579,oot579
        data=pd.read_csv(ele)
        d.append(data)
#train = dt.fread(baseLoc+"baseDatasets/train.jay")
# train=train.to_pandas()#[0:100000]
#len(set(d[0]['user_id'].unique()).intersection(set(d[1]['user_id'].unique()))),len(d[1]['user_id'].unique())
#d[1].head()
#d[2][d[2]['user_id']==480368]
train_df=d[0]

In [ ]:
%%time
def breakinUsers(r):
    return (r['content_id'].values,r['answered_correctly'].values,r['timestamp'].values,
            #r['lect_count'].values,r['relevantLectattended'].values,r['quest_roll_average_50'].values,
           r['tag0'].values,r['tag1'].values,r['tag2'].values,r['tag3'].values,r['tag4'].values,r['tag5'].values,
           r['part'].values,r['bundle_id'].values)
group = train_df.groupby('user_id').apply(breakinUsers)
# group = train_df[['user_id', 'content_id', 'answered_correctly','timestamp']].groupby('user_id').apply(lambda r: (
    
#             r['content_id'].values,
#             r['answered_correctly'].values,r['timestamp'].values))
del train_df##########################change
gc.collect()
lectureDict1=  createl2conv()
#train_df[train_df['content_id']==13522].sort_values(['content_id','row_id'])

In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.samples = group
        
#         self.user_ids = [x for x in group.index]
        self.user_ids = []
        for user_id in group.index:
            tup= group[user_id]
            #q, qa ,_=tup[0],tup[1],tup[2]
            if len(tup[0]) < 2: #HDKIM 10
                continue
            self.user_ids.append(user_id)
            
            #HDKIM Memory reduction
            #if len(q)>self.max_seq:
            #    group[user_id] = (q[-self.max_seq:],qa[-self.max_seq:])

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        tup = self.samples[user_id]
        feature=11#len(tup[])##################chnage
        #q_, qa_,times=tup[0],tup[1],tup[2]
        seq_len = len(tup[0])

        q = np.zeros(self.max_seq, dtype=int)
        qa = -1*np.ones(self.max_seq, dtype=int)
        outs=[q,qa]
        for i in range(2,feature):
            x=np.zeros(self.max_seq, dtype=float)
            outs.append(x)
        
        
        if seq_len >= self.max_seq:
            #HDKIM
            if random.random()>0.95:
                start = random.randint(0,(seq_len-self.max_seq))
                end = start + self.max_seq
                for i in range(feature):
                    #print(i)
                    outs[i][:]= tup[i][start:end]

            else:
                for i in range(feature):
                    #print(i)
                    outs[i][:] = tup[i][-self.max_seq:]

        else:
            #HDKIM
            if random.random()>0.95:
                #HDKIMHDKIM
                start = 0
                end = random.randint(2,seq_len)
                seq_len = end - start
                for i in range(feature):
                    #print(i)
                    outs[i][-seq_len:] = tup[i][0:seq_len]

            else:
                for i in range(feature):
                    #print(i)
                    outs[i][-seq_len:] =  tup[i]
 

        target_id = outs[0][1:]
        label = outs[1][1:]
        label,weights=np.where(label==-1,0,label),np.where(label==-1,0,1)
        #print(np.unique(label))
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill
#         t=outs[2][1:]
#         lect_count=outs[3][1:]
#         relevantLectattended=outs[4][1:]
#         quest_roll_average_50=outs[5][1:]
        ret=[x, target_id, label,weights]
        for i in range(2,feature):
            ret.append(outs[i][1:])
            
        
        #`print(tuple(ret))
        return tuple(ret)#x, target_id, label,weights,t,lect_count,relevantLectattended,quest_roll_average_50
    #train_df[train_df['user_id']==115]

In [ ]:
dataset = SAKTDataset(group, n_skill)
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=8)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
#         for i in range(pe.shape[1]):
#             print(pe[:, i])
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        #print(x.shape,self.pe.shape)
        x = x + self.pe.permute(1, 0, 2)
        return self.dropout(x)

class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)
class FFN0(nn.Module):
    def __init__(self, state_size=28):
        super(FFN0, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(4, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.layer_normal = nn.LayerNorm(state_size) 
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        x=self.layer_normal(x)
        #print(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128): #HDKIM 100->MAX_SEQ
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim
        self.extraFeature=9
        adjung=48-3
        self.embedding = nn.Embedding(2*n_skill+1, embed_dim-adjung)
        self.pos_embedding = PositionalEncoding(128, max_len=159)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim-adjung)
        self.t_embedding = nn.Embedding(200, 4)
        self.p_embedding = nn.Embedding(10, 4)
        self.b_embedding = nn.Embedding(n_skill+1, 16)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim)# +28
        self.layer_normal2 = nn.LayerNorm(embed_dim+adjung)# +28

        self.ffn = FFN(embed_dim)
        self.ffn0 = FFN0()
        self.pred = nn.Linear(embed_dim, 1)#+28
    
    def forward(self, x, question_ids,*args):
        see=False
        outs=[]
        loop=0
        for i in args:
            if loop<1:outs.append(i.unsqueeze(2))
            else:outs.append(i)
            loop+=1
       
        #times=times.unsqueeze(2)
        device = x.device
        #print(x[0])
        #print(question_ids[0])
        if see:print(x.shape,question_ids.shape,23)
        x = self.embedding(x)
        e = self.e_embedding(question_ids)
        #pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        #print(x.shape,a.shape,b.shape,c.shape,d.shape)
        if False:
            y=torch.cat(
                (outs[0],outs[1],outs[2],outs[3],), axis=2)
            z=self.ffn0(y)
            if see:print(x.shape,z.shape,9)
        
            
        if True:
            for i in range(self.extraFeature):
                if i>=1:
                    temp0 = torch.tensor(outs[i]).to(torch.int64)

                if i==7:
                    temp=self.p_embedding(temp0)
                    #e = torch.cat((e,temp,), axis=2)
                elif i==8:
                    temp=self.b_embedding(temp0)
                    #e = torch.cat((e,temp,), axis=2)
                elif i>=1 :
                    temp=self.t_embedding(temp0)
                    #print(temp.shape,e.shape,24)
                    #e = torch.cat((e,temp,), axis=2)
                else :
                    temp=outs[i]
                x = torch.cat((x,temp,), axis=2)
                e = torch.cat((e,temp,), axis=2)
                
                
                
                    
        elif False:
            x = torch.cat(
                (x,z,), axis=2)
            #print(x.shape,z.shape)
            
                
#         else:
#             x = torch.cat(
#                 (x,outs[0],outs[1],outs[2],outs[3],), axis=2)#xself.pos_embedding(x)
        #print(x[0][0])
        #print(x.shape)
        #x = x + pos_x
        
        
        
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        #print(x.shape,e.shape,234)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        if see:print(att_output.shape,z.shape)
        
        #print(x[0][0])
        if see:print(att_output.shape,z.shape,1)
        att_output = self.layer_normal(att_output +e)#+additona+ e
        if see:print(att_output.shape,z.shape,2)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]
        if see:print(att_output.shape,3)
        if False:
            att_output = torch.cat(
                (att_output,y,), axis=2)
        if see:print(y[0],12)#,att_output)
        x = self.ffn(att_output)
        if see:print(att_output.shape,x.shape,4)
        #x = self.layer_normal2(x +att_output)#x 
        x = self.pred(x)
        #print(att_weight[0][0])
        
        return x.squeeze(-1), att_weight

In [ ]:
import math
n_skill =14500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=SAKTModel(n_skill, embed_dim=128)

model.load_state_dict(torch.load("/kaggle/input/model3/SAKT-HDKIM (2).pt",map_location=torch.device('cpu')))
model.eval()

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model.to(device)


In [ ]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()
        weights=item[3].to(device).float()
        extra=[]
        for i in range(4,4+model.extraFeature):
            extra.append(item[i].to(device).float())
        extra=tuple(extra)
        #print(weights.shape)

        optim.zero_grad()
        #print(extra)
        output, atten_weight = model(x, target_id,*extra)
        #print(output.shape, label.shape)
        #weights=torch.tensor(np.zeros(shape=output.shape))
        criterion = nn.BCEWithLogitsLoss(weight=weights)#weight=weights
        criterion.to(device)
        loss = criterion(output, label)#
        loss.backward()
        optim.step()
        train_loss.append(loss.item())
        #print(loss)
        output = output[:, -1]
        label = label[:, -1] 
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [ ]:
#BASE 6M 40 0.761 80 0.787  120 0.812 #new 40 0.762 80:0.785  120 0.808
epochs =1 #HDKIM 20 with 1 million accounts epoch - 34 train_loss - 0.23 acc - 0.609 auc - 0.647
#base on 6 million 20 epoch is 0.72 auc. added mask on irrelvant prefex sequesnce and auc is 0.739
#on full daset add mask doesnt hav any difference
dataset = SAKTDataset(group, n_skill)
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=8)
del group
for epoch in range(epochs):
    loss, acc, auc = train_epoch(model, dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, loss, acc, auc))

In [ ]:
#validation check
#grp=group
#group=grp.copy()
if False:#validation data split
    valids=[]
    dev=d[1].sort_values('timestamp')
    batch=int(dev.shape[0]/10)
    for i in range(batch):
        valids.append(dev[i*batch:(i+1)*batch])
    
if True:
    grpLen=len(group[115])
    #on all  new
    #test_df=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', usecols=[1, 2, 3, 4, 7], dtype=dtype,nrows=10000000)[-100000:]
    #om seggregated
    prev_test_df=None
    for test_df in valids:
        if (prev_test_df is not None) :
            prev_test_df = prev_test_df#[prev_test_df.content_type_id == False]
            prev_group  = prev_test_df.groupby('user_id').apply(breakinUsers)
        for prev_user_id in prev_group.index:
            case=False
            if prev_user_id in group.index :case=True
            tup=[]
            for i in range(grpLen):
                if case:
                    tup.append(np.append(group[prev_user_id][i],prev_group[prev_user_id][i]))
                else:
                    tup.append(prev_group[prev_user_id][i])

            if len(group[prev_user_id][0])>MAX_SEQ:
                for i in range(grpLen):tup[i]= tup[i][-MAX_SEQ:]
            group[prev_user_id]=tuple(tup)

    #prev_test_df = test_df.copy()
        #test_df['timestamp']=test_df['timestamp']/maxTimestamp
        test_df = test_df[test_df.content_type_id == False]
        actual=test_df['answered_correctly']
    
    
    #test_df.to_csv(str(loop)+".csv")
    test_dataset = TestDataset(group, test_df, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=4096, shuffle=False)
    
    
    outs = []
 
    for item in tqdm(test_dataloader):
        #print("done")
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        #weights=item[3].to(device).float()
        times=item[2].to(device).float()
        

        with torch.no_grad():
            output, att_weight = model(x, target_id,times)
        
        
        output = torch.sigmoid(output)
        output = output[:, -1]

        # pred = (output >= 0.5).long()
        # loss = criterion(output, label)

        # val_loss.append(loss.item())
        # num_corrects += (pred == label).sum().item()
        # num_total += len(label)

        # labels.extend(label.squeeze(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())
        
    #print(type(outs),outs)
    test_df['answered_correctly'] =  outs#[0.2 for c in outs ]
    print(roc_auc_score(actual,test_df['answered_correctly'] ))